In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HGTConv, global_mean_pool
import os
import scipy.io as sio
import numpy as np

def load_mat_samples(mat_file_path):
    """
    Load all sliding window samples from a single .mat file (10240x20)
    """
    mat_data = sio.loadmat(mat_file_path)
    samples = []
    for key in mat_data:
        if key.startswith('sample_') and isinstance(mat_data[key], np.ndarray):
            samples.append(mat_data[key])
    return samples

def extract_numeric_label(filename):
    """
    Extract the number at the beginning of the filename as the category label
    For example: ‘3_parallel_misalignment_060mm_500.mat’ → 3
    """
    return int(filename.split('_')[0])

def load_all_mat_folder(folder_path):
    all_data = []
    all_labels = []
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.mat')]

    # Sort filenames in numerical order
    file_list.sort(key=lambda x: int(x.split('_')[0]))

    for f in file_list:
        file_path = os.path.join(folder_path, f)
        samples = load_mat_samples(file_path)

        numeric_label = int(f.split('_')[0])  # Automatically extract numeric tags
        all_data.extend(samples)
        all_labels.extend([numeric_label] * len(samples))

    return all_data, all_labels



In [ ]:
folder_path1 = "G:\XX\XX"  # Replace with the actual path
data1, labels1 = load_all_mat_folder(folder_path1)

folder_path2 = "G:\XX\XXX"  # Replace with the actual path
data2, labels2 = load_all_mat_folder(folder_path2)

folder_path3 = "G:\XX\XXX"  # Replace with the actual path
data3, labels3 = load_all_mat_folder(folder_path3)

print(f"Total sample size: {len(data1)}")
print(f"First group of samples shape: {data1[0].shape}")  
print(f"First set of tags: {labels1[0]}")  



In [ ]:
import os
import torch
from torch_geometric.data import HeteroData
import numpy as np
from scipy.signal import hilbert
from scipy.stats import kurtosis
from numpy.fft import fft
from collections import defaultdict

# Save Path
save_path = r"G:\XXXX\XXXXX\XXXXXX"
os.makedirs(save_path, exist_ok=True)

# Configuration
freq_bands = [50, 100, 150, 200, 600, 1200, 1800]
jump_edges = [(0, 2), (0, 3), (0, 4), (4, 6)]
modality_map = {
    'displacement': range(0, 4),
    'acceleration': range(4, 10),
    'sound': range(10, 14),
    'blade_stress': range(14, 16),
    'casing_stress': range(16, 20)
}
modality_to_id = {k: i for i, k in enumerate(modality_map)}

def channel_type(c):
    for mod, ch_range in modality_map.items():
        if c in ch_range:
            return mod
    return 'unknown'

def extract_features(signal, fs=10240):
    rms = np.sqrt(np.mean(signal ** 2))
    k = kurtosis(signal)
    env = np.abs(hilbert(signal))
    peak_env = np.max(env)

    N = len(signal)
    freq = np.fft.fftfreq(N, d=1/fs)[:N//2]
    fft_vals = np.abs(fft(signal))[:N//2]
    psd = fft_vals ** 2
    psd /= np.sum(psd + 1e-12)

    entropy = -np.sum(psd * np.log(psd + 1e-12))
    dominant_idx = np.argmax(fft_vals)
    dominant_freq = freq[dominant_idx]
    flatness = np.exp(np.mean(np.log(fft_vals + 1e-12))) / (np.mean(fft_vals) + 1e-12)
    top3 = np.sort(fft_vals)[-3:]
    top3_ratio = np.sum(top3) / (np.sum(fft_vals) + 1e-12)

    return [rms, k, peak_env, entropy, dominant_freq, flatness, top3_ratio], fft_vals, freq

def build_graph_from_sample(sample, label, idx, save_path, domain_id):
    node_lists = defaultdict(list)
    node_features = defaultdict(list)
    freq_ids = defaultdict(list)
    edge_index_dict = defaultdict(lambda: [[], []])
    node_counter = defaultdict(int)

    # === Build Node ===
    for ch in range(sample.shape[1]):
        mod = channel_type(ch)
        signal = sample[:, ch]
        stat_feats, fft_vals, freqs = extract_features(signal)

        for f_idx, f in enumerate(freq_bands):
            band_mask = (freqs >= f - 10) & (freqs <= f + 10)
            band_vals = fft_vals[band_mask]
            band_vals = np.interp(np.linspace(0, len(band_vals)-1, 21), np.arange(len(band_vals)), band_vals)
            feature = torch.tensor(np.concatenate([stat_feats, band_vals]), dtype=torch.float)

            nid = node_counter[mod]
            node_counter[mod] += 1
            node_lists[mod].append(nid)
            node_features[mod].append(feature)
            freq_ids[mod].append(f_idx)

    # === Frequency Edge (freq_edge): Same mode, same frequency band, different channels ===
    for mod in node_lists:
        group = defaultdict(list)
        for i, fidx in enumerate(freq_ids[mod]):
            group[fidx].append(node_lists[mod][i])
        for fidx, nodes in group.items():
            for i in range(len(nodes)):
                for j in range(i + 1, len(nodes)):
                    edge_index_dict[(mod, 'freq_edge', mod)][0].extend([nodes[i], nodes[j]])
                    edge_index_dict[(mod, 'freq_edge', mod)][1].extend([nodes[j], nodes[i]])

    # === Frequency-hopping edge (hop_edge): Same mode, same channel, different frequency bands ===
    for mod in node_lists:
        f_map = defaultdict(list)
        for i, fidx in enumerate(freq_ids[mod]):
            f_map[fidx].append(node_lists[mod][i])
        for i, j in jump_edges:
            if i in f_map and j in f_map:
                for u in f_map[i]:
                    for v in f_map[j]:
                        edge_index_dict[(mod, 'hop_edge', mod)][0].append(u)
                        edge_index_dict[(mod, 'hop_edge', mod)][1].append(v)

    # === Modality Edge: Same frequency band, cross-modality ===
    for f_idx in range(len(freq_bands)):
        f_nodes = []
        for mod in node_lists:
            for i, f in enumerate(freq_ids[mod]):
                if f == f_idx:
                    f_nodes.append((mod, node_lists[mod][i]))
        for i in range(len(f_nodes)):
            for j in range(i+1, len(f_nodes)):
                m1, u = f_nodes[i]
                m2, v = f_nodes[j]
                if m1 != m2:
                    edge_index_dict[(m1, 'modality_edge', m2)][0].append(u)
                    edge_index_dict[(m1, 'modality_edge', m2)][1].append(v)

    # === Constructing Graph Data Structures ===
    data = HeteroData()
    for mod in node_lists:
        x = torch.stack(node_features[mod])
        data[mod].x = x
        data[mod].freq_ids = torch.tensor(freq_ids[mod], dtype=torch.long)
        data[mod].modality_ids = torch.full((len(x),), modality_to_id[mod], dtype=torch.long)
        data[mod].node_ids = torch.arange(len(x))
        data[mod].domain_ids = torch.full((len(x),), domain_id, dtype=torch.long)

    # === Modal Supernode (supernode_edge) ===
    for mod in node_lists:
        feat = torch.mean(data[mod].x, dim=0, keepdim=True)
        super_id = len(data[mod].x)
        data[mod].x = torch.cat([data[mod].x, feat], dim=0)
        data[mod].freq_ids = torch.cat([data[mod].freq_ids, torch.tensor([-1])])
        data[mod].modality_ids = torch.cat([data[mod].modality_ids, torch.tensor([modality_to_id[mod]])])
        data[mod].node_ids = torch.cat([data[mod].node_ids, torch.tensor([super_id])])
        data[mod].domain_ids = torch.cat([data[mod].domain_ids, torch.tensor([domain_id])])
        for i in range(super_id):
            edge_index_dict[(mod, 'supernode_edge', mod)][0].append(i)
            edge_index_dict[(mod, 'supernode_edge', mod)][1].append(super_id)

    # === Spatial Adjacency Edge (inter_supernode_edge) ===
    mods = list(node_lists.keys())
    for i in range(len(mods)):
        for j in range(i+1, len(mods)):
            m1, m2 = mods[i], mods[j]
            nid1 = len(data[m1].x) - 1
            nid2 = len(data[m2].x) - 1
            edge_index_dict[(m1, 'inter_supernode_edge', m2)][0].append(nid1)
            edge_index_dict[(m1, 'inter_supernode_edge', m2)][1].append(nid2)

    # === Save Edge ===
    for (s, e, t), (src, dst) in edge_index_dict.items():
        data[(s, e, t)].edge_index = torch.tensor([src, dst], dtype=torch.long)

    # === Label Information ===
    data.y = torch.tensor([label], dtype=torch.long)
    data.domain_id = torch.tensor([domain_id], dtype=torch.long)

    # === Store graph data ===
    torch.save(data, os.path.join(save_path, f"{idx}_{label}.pt"))



# Rebuild all map data
for i in range(len(data1)):
    build_graph_from_sample(data1[i], labels1[i], i, save_path, domain_id=0)  # Add domain_id


print("All diagrams have been reconstructed and saved.")

In [ ]:
import os
import torch
from torch_geometric.data import HeteroData
import numpy as np
from scipy.signal import hilbert
from scipy.stats import kurtosis
from numpy.fft import fft
from collections import defaultdict

#  Save Path
save_path = r"G:\XXXX\XXXXX\XXXXXX2"
os.makedirs(save_path, exist_ok=True)

# Configuration
freq_bands = [50, 100, 150, 200, 600, 1200, 1800]
jump_edges = [(0, 2), (0, 3), (0, 4), (4, 6)]
modality_map = {
    'displacement': range(0, 4),
    'acceleration': range(4, 10),
    'sound': range(10, 14),
    'blade_stress': range(14, 16),
    'casing_stress': range(16, 20)
}
modality_to_id = {k: i for i, k in enumerate(modality_map)}

def channel_type(c):
    for mod, ch_range in modality_map.items():
        if c in ch_range:
            return mod
    return 'unknown'

def extract_features(signal, fs=10240):
    rms = np.sqrt(np.mean(signal ** 2))
    k = kurtosis(signal)
    env = np.abs(hilbert(signal))
    peak_env = np.max(env)

    N = len(signal)
    freq = np.fft.fftfreq(N, d=1/fs)[:N//2]
    fft_vals = np.abs(fft(signal))[:N//2]
    psd = fft_vals ** 2
    psd /= np.sum(psd + 1e-12)

    entropy = -np.sum(psd * np.log(psd + 1e-12))
    dominant_idx = np.argmax(fft_vals)
    dominant_freq = freq[dominant_idx]
    flatness = np.exp(np.mean(np.log(fft_vals + 1e-12))) / (np.mean(fft_vals) + 1e-12)
    top3 = np.sort(fft_vals)[-3:]
    top3_ratio = np.sum(top3) / (np.sum(fft_vals) + 1e-12)

    return [rms, k, peak_env, entropy, dominant_freq, flatness, top3_ratio], fft_vals, freq

def build_graph_from_sample(sample, label, idx, save_path, domain_id):
    node_lists = defaultdict(list)
    node_features = defaultdict(list)
    freq_ids = defaultdict(list)
    edge_index_dict = defaultdict(lambda: [[], []])
    node_counter = defaultdict(int)

    # === Build Node ===
    for ch in range(sample.shape[1]):
        mod = channel_type(ch)
        signal = sample[:, ch]
        stat_feats, fft_vals, freqs = extract_features(signal)

        for f_idx, f in enumerate(freq_bands):
            band_mask = (freqs >= f - 10) & (freqs <= f + 10)
            band_vals = fft_vals[band_mask]
            band_vals = np.interp(np.linspace(0, len(band_vals)-1, 21), np.arange(len(band_vals)), band_vals)
            feature = torch.tensor(np.concatenate([stat_feats, band_vals]), dtype=torch.float)

            nid = node_counter[mod]
            node_counter[mod] += 1
            node_lists[mod].append(nid)
            node_features[mod].append(feature)
            freq_ids[mod].append(f_idx)

    # === Frequency Edge (freq_edge): Same mode, same frequency band, different channels ===
    for mod in node_lists:
        group = defaultdict(list)
        for i, fidx in enumerate(freq_ids[mod]):
            group[fidx].append(node_lists[mod][i])
        for fidx, nodes in group.items():
            for i in range(len(nodes)):
                for j in range(i + 1, len(nodes)):
                    edge_index_dict[(mod, 'freq_edge', mod)][0].extend([nodes[i], nodes[j]])
                    edge_index_dict[(mod, 'freq_edge', mod)][1].extend([nodes[j], nodes[i]])

    # === Frequency-hopping edge (hop_edge): Same mode, same channel, different frequency bands ===
    for mod in node_lists:
        f_map = defaultdict(list)
        for i, fidx in enumerate(freq_ids[mod]):
            f_map[fidx].append(node_lists[mod][i])
        for i, j in jump_edges:
            if i in f_map and j in f_map:
                for u in f_map[i]:
                    for v in f_map[j]:
                        edge_index_dict[(mod, 'hop_edge', mod)][0].append(u)
                        edge_index_dict[(mod, 'hop_edge', mod)][1].append(v)

    # === Modality Edge: Same frequency band, cross-modality ===
    for f_idx in range(len(freq_bands)):
        f_nodes = []
        for mod in node_lists:
            for i, f in enumerate(freq_ids[mod]):
                if f == f_idx:
                    f_nodes.append((mod, node_lists[mod][i]))
        for i in range(len(f_nodes)):
            for j in range(i+1, len(f_nodes)):
                m1, u = f_nodes[i]
                m2, v = f_nodes[j]
                if m1 != m2:
                    edge_index_dict[(m1, 'modality_edge', m2)][0].append(u)
                    edge_index_dict[(m1, 'modality_edge', m2)][1].append(v)

    # === Constructing Graph Data Structures ===
    data = HeteroData()
    for mod in node_lists:
        x = torch.stack(node_features[mod])
        data[mod].x = x
        data[mod].freq_ids = torch.tensor(freq_ids[mod], dtype=torch.long)
        data[mod].modality_ids = torch.full((len(x),), modality_to_id[mod], dtype=torch.long)
        data[mod].node_ids = torch.arange(len(x))
        data[mod].domain_ids = torch.full((len(x),), domain_id, dtype=torch.long)

    # === Modal Supernode (supernode_edge) ===
    for mod in node_lists:
        feat = torch.mean(data[mod].x, dim=0, keepdim=True)
        super_id = len(data[mod].x)
        data[mod].x = torch.cat([data[mod].x, feat], dim=0)
        data[mod].freq_ids = torch.cat([data[mod].freq_ids, torch.tensor([-1])])
        data[mod].modality_ids = torch.cat([data[mod].modality_ids, torch.tensor([modality_to_id[mod]])])
        data[mod].node_ids = torch.cat([data[mod].node_ids, torch.tensor([super_id])])
        data[mod].domain_ids = torch.cat([data[mod].domain_ids, torch.tensor([domain_id])])
        for i in range(super_id):
            edge_index_dict[(mod, 'supernode_edge', mod)][0].append(i)
            edge_index_dict[(mod, 'supernode_edge', mod)][1].append(super_id)

    # === Spatial Adjacency Edge (inter_supernode_edge) ===
    mods = list(node_lists.keys())
    for i in range(len(mods)):
        for j in range(i+1, len(mods)):
            m1, m2 = mods[i], mods[j]
            nid1 = len(data[m1].x) - 1
            nid2 = len(data[m2].x) - 1
            edge_index_dict[(m1, 'inter_supernode_edge', m2)][0].append(nid1)
            edge_index_dict[(m1, 'inter_supernode_edge', m2)][1].append(nid2)

    # === Save Edge ===
    for (s, e, t), (src, dst) in edge_index_dict.items():
        data[(s, e, t)].edge_index = torch.tensor([src, dst], dtype=torch.long)

    # === Label Information ===
    data.y = torch.tensor([label], dtype=torch.long)
    data.domain_id = torch.tensor([domain_id], dtype=torch.long)

    # === Store graph data ===
    torch.save(data, os.path.join(save_path, f"{idx}_{label}.pt"))



# Rebuild all map data
for i in range(len(data2)):
    build_graph_from_sample(data2[i], labels2[i], i, save_path, domain_id=1)  # Add domain_id


print("All diagrams have been reconstructed and saved.")

In [ ]:
import os
import torch
from torch_geometric.data import HeteroData
import numpy as np
from scipy.signal import hilbert
from scipy.stats import kurtosis
from numpy.fft import fft
from collections import defaultdict

# Save Path
save_path = r"G:\XXXX\XXXXX\XXXXXX3"
os.makedirs(save_path, exist_ok=True)

# Configuration
freq_bands = [50, 100, 150, 200, 600, 1200, 1800]
jump_edges = [(0, 2), (0, 3), (0, 4), (4, 6)]
modality_map = {
    'displacement': range(0, 4),
    'acceleration': range(4, 10),
    'sound': range(10, 14),
    'blade_stress': range(14, 16),
    'casing_stress': range(16, 20)
}
modality_to_id = {k: i for i, k in enumerate(modality_map)}

def channel_type(c):
    for mod, ch_range in modality_map.items():
        if c in ch_range:
            return mod
    return 'unknown'

def extract_features(signal, fs=10240):
    rms = np.sqrt(np.mean(signal ** 2))
    k = kurtosis(signal)
    env = np.abs(hilbert(signal))
    peak_env = np.max(env)

    N = len(signal)
    freq = np.fft.fftfreq(N, d=1/fs)[:N//2]
    fft_vals = np.abs(fft(signal))[:N//2]
    psd = fft_vals ** 2
    psd /= np.sum(psd + 1e-12)

    entropy = -np.sum(psd * np.log(psd + 1e-12))
    dominant_idx = np.argmax(fft_vals)
    dominant_freq = freq[dominant_idx]
    flatness = np.exp(np.mean(np.log(fft_vals + 1e-12))) / (np.mean(fft_vals) + 1e-12)
    top3 = np.sort(fft_vals)[-3:]
    top3_ratio = np.sum(top3) / (np.sum(fft_vals) + 1e-12)

    return [rms, k, peak_env, entropy, dominant_freq, flatness, top3_ratio], fft_vals, freq

def build_graph_from_sample(sample, label, idx, save_path, domain_id):
    node_lists = defaultdict(list)
    node_features = defaultdict(list)
    freq_ids = defaultdict(list)
    edge_index_dict = defaultdict(lambda: [[], []])
    node_counter = defaultdict(int)

    # === Build Node ===
    for ch in range(sample.shape[1]):
        mod = channel_type(ch)
        signal = sample[:, ch]
        stat_feats, fft_vals, freqs = extract_features(signal)

        for f_idx, f in enumerate(freq_bands):
            band_mask = (freqs >= f - 10) & (freqs <= f + 10)
            band_vals = fft_vals[band_mask]
            band_vals = np.interp(np.linspace(0, len(band_vals)-1, 21), np.arange(len(band_vals)), band_vals)
            feature = torch.tensor(np.concatenate([stat_feats, band_vals]), dtype=torch.float)

            nid = node_counter[mod]
            node_counter[mod] += 1
            node_lists[mod].append(nid)
            node_features[mod].append(feature)
            freq_ids[mod].append(f_idx)

    # === Frequency Edge (freq_edge): Same mode, same frequency band, different channels ===
    for mod in node_lists:
        group = defaultdict(list)
        for i, fidx in enumerate(freq_ids[mod]):
            group[fidx].append(node_lists[mod][i])
        for fidx, nodes in group.items():
            for i in range(len(nodes)):
                for j in range(i + 1, len(nodes)):
                    edge_index_dict[(mod, 'freq_edge', mod)][0].extend([nodes[i], nodes[j]])
                    edge_index_dict[(mod, 'freq_edge', mod)][1].extend([nodes[j], nodes[i]])

    # === Frequency-hopping edge (hop_edge): Same mode, same channel, different frequency bands ===
    for mod in node_lists:
        f_map = defaultdict(list)
        for i, fidx in enumerate(freq_ids[mod]):
            f_map[fidx].append(node_lists[mod][i])
        for i, j in jump_edges:
            if i in f_map and j in f_map:
                for u in f_map[i]:
                    for v in f_map[j]:
                        edge_index_dict[(mod, 'hop_edge', mod)][0].append(u)
                        edge_index_dict[(mod, 'hop_edge', mod)][1].append(v)

    # === Modality Edge: Same frequency band, cross-modality ===
    for f_idx in range(len(freq_bands)):
        f_nodes = []
        for mod in node_lists:
            for i, f in enumerate(freq_ids[mod]):
                if f == f_idx:
                    f_nodes.append((mod, node_lists[mod][i]))
        for i in range(len(f_nodes)):
            for j in range(i+1, len(f_nodes)):
                m1, u = f_nodes[i]
                m2, v = f_nodes[j]
                if m1 != m2:
                    edge_index_dict[(m1, 'modality_edge', m2)][0].append(u)
                    edge_index_dict[(m1, 'modality_edge', m2)][1].append(v)

    # === Constructing Graph Data Structures ===
    data = HeteroData()
    for mod in node_lists:
        x = torch.stack(node_features[mod])
        data[mod].x = x
        data[mod].freq_ids = torch.tensor(freq_ids[mod], dtype=torch.long)
        data[mod].modality_ids = torch.full((len(x),), modality_to_id[mod], dtype=torch.long)
        data[mod].node_ids = torch.arange(len(x))
        data[mod].domain_ids = torch.full((len(x),), domain_id, dtype=torch.long)

    # === Modal Supernode (supernode_edge) ===
    for mod in node_lists:
        feat = torch.mean(data[mod].x, dim=0, keepdim=True)
        super_id = len(data[mod].x)
        data[mod].x = torch.cat([data[mod].x, feat], dim=0)
        data[mod].freq_ids = torch.cat([data[mod].freq_ids, torch.tensor([-1])])
        data[mod].modality_ids = torch.cat([data[mod].modality_ids, torch.tensor([modality_to_id[mod]])])
        data[mod].node_ids = torch.cat([data[mod].node_ids, torch.tensor([super_id])])
        data[mod].domain_ids = torch.cat([data[mod].domain_ids, torch.tensor([domain_id])])
        for i in range(super_id):
            edge_index_dict[(mod, 'supernode_edge', mod)][0].append(i)
            edge_index_dict[(mod, 'supernode_edge', mod)][1].append(super_id)

    # === Spatial Adjacency Edge (inter_supernode_edge) ===
    mods = list(node_lists.keys())
    for i in range(len(mods)):
        for j in range(i+1, len(mods)):
            m1, m2 = mods[i], mods[j]
            nid1 = len(data[m1].x) - 1
            nid2 = len(data[m2].x) - 1
            edge_index_dict[(m1, 'inter_supernode_edge', m2)][0].append(nid1)
            edge_index_dict[(m1, 'inter_supernode_edge', m2)][1].append(nid2)

    # === Save Edge ===
    for (s, e, t), (src, dst) in edge_index_dict.items():
        data[(s, e, t)].edge_index = torch.tensor([src, dst], dtype=torch.long)

    # === Label Information ===
    data.y = torch.tensor([label], dtype=torch.long)
    data.domain_id = torch.tensor([domain_id], dtype=torch.long)

    # === Store graph data ===
    torch.save(data, os.path.join(save_path, f"{idx}_{label}.pt"))



# Rebuild all map data
for i in range(len(data3)):
    build_graph_from_sample(data3[i], labels3[i], i, save_path, domain_id=2)  # Add domain_id


print("All diagrams have been reconstructed and saved.")